In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


data = pd.read_csv('Breast_Cancer.csv')  

# Convert 'A Stage' to integers using LabelEncoder
label_encoder = LabelEncoder()
data['A Stage'] = label_encoder.fit_transform(data['A Stage'])

# Divide features into numeric and categorical ones, excluding 'A Stage'
numeric_features = list(data.select_dtypes(include=['int64', 'float64']).columns)

# Ensure 'A Stage' is not in the numeric features list
if 'A Stage' in numeric_features:
    numeric_features.remove('A Stage')

categorical_features = [item for item in data.columns if item not in numeric_features and item != 'A Stage']

# Encode other categorical features and handle missing values
data_encoded = data.copy()

# Example encoding step for one of the categorical features
data_encoded['Status'] = data_encoded['Status'].astype(pd.CategoricalDtype(categories=['Alive', 'Dead'], ordered=True))
data_encoded['Status'] = data_encoded['Status'].cat.codes

# Update feature types after encoding
numeric_features = list(data_encoded.select_dtypes(include=['int8', 'int64', 'float64']).columns)

# Ensure 'A Stage' is not in the numeric features list
if 'A Stage' in numeric_features:
    numeric_features.remove('A Stage')

categorical_features = [item for item in data_encoded.columns if item not in numeric_features and item != 'A Stage']

# Create transformers for data preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Processing of missing values
    ('scaler', StandardScaler())  # Scaling of numerical features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Processing of missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Converting categorical variables to binary numeric ones
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply transformers to the data, excluding 'A Stage'
data_preprocessed = preprocessor.fit_transform(data_encoded)

# Get the encoded categorical feature names after one-hot encoding
encoded_categorical_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(input_features=categorical_features)

# Combine numeric and categorical feature names, and add 'A Stage' at the end
encoded_feature_names = numeric_features + list(encoded_categorical_feature_names) + ['A Stage']

# Add 'A Stage' back to the preprocessed data
data_preprocessed = np.hstack((data_preprocessed, data_encoded[['A Stage']].values))

# Verify the shapes match
print(f"Shape of data_preprocessed: {data_preprocessed.shape}")
print(f"Number of encoded feature names: {len(encoded_feature_names)}")

# Check details of generated features
print(f"Numeric features after preprocessing: {numeric_features}")
print(f"Categorical features after one-hot encoding: {list(encoded_categorical_feature_names)}")

# Create the DataFrame
if data_preprocessed.shape[1] == len(encoded_feature_names):
    data_preprocessed_df = pd.DataFrame(data_preprocessed, columns=encoded_feature_names)
    print(data_preprocessed_df.head())
else:
    print(f"Mismatch! Data shape: {data_preprocessed.shape}, Feature names length: {len(encoded_feature_names)}")

# Creating a DataFrame from encoded data
data_preprocessed_df = pd.DataFrame(data_preprocessed, columns=encoded_feature_names)
print(data_preprocessed_df.head())


Shape of data_preprocessed: (4024, 39)
Number of encoded feature names: 39
Numeric features after preprocessing: ['Age', 'Tumor Size', 'Regional Node Examined', 'Reginol Node Positive', 'Survival Months', 'Status']
Categorical features after one-hot encoding: ['Race_Black', 'Race_Other', 'Race_White', 'Marital Status_Divorced', 'Marital Status_Married', 'Marital Status_Separated', 'Marital Status_Single ', 'Marital Status_Widowed', 'T Stage _T1', 'T Stage _T2', 'T Stage _T3', 'T Stage _T4', 'N Stage_N1', 'N Stage_N2', 'N Stage_N3', '6th Stage_IIA', '6th Stage_IIB', '6th Stage_IIIA', '6th Stage_IIIB', '6th Stage_IIIC', 'differentiate_Moderately differentiated', 'differentiate_Poorly differentiated', 'differentiate_Undifferentiated', 'differentiate_Well differentiated', 'Grade_ anaplastic; Grade IV', 'Grade_1', 'Grade_2', 'Grade_3', 'Estrogen Status_Negative', 'Estrogen Status_Positive', 'Progesterone Status_Negative', 'Progesterone Status_Positive']
        Age  Tumor Size  Regional Nod

In [5]:
data_preprocessed_df

,Age,Tumor Size,Regional Node Examined,Reginol Node Positive,Survival Months,Status,Race_Black,Race_Other,Race_White,Marital Status_Divorced,...,differentiate_Well differentiated,Grade_ anaplastic; Grade IV,Grade_1,Grade_2,Grade_3,Estrogen Status_Negative,Estrogen Status_Positive,Progesterone Status_Negative,Progesterone Status_Positive,A Stage
0,1.565253,-1.253661,1.190676,-0.618172,-0.492961,-0.425148,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
1,-0.443222,0.214345,-0.044095,0.164807,-0.405695,-0.425148,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
2,0.449434,1.540287,-0.044095,0.556296,0.161530,-0.425148,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,0.449434,-0.590691,-1.525820,-0.618172,0.554224,-0.425148,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,-0.777968,0.498475,-1.402343,-0.618172,-0.929288,-0.425148,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,0.895761,-1.016886,-1.649297,-0.618172,-0.972921,-0.425148,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
4020,0.226270,0.735251,-0.044095,0.752041,-0.100266,-0.425148,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
4021,1.565253,-0.401271,-0.414526,-0.226682,-0.100266,-0.425148,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
4022,0.449434,0.640541,-0.414526,-0.618172,0.030632,-0.425148,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Custom neural network with Fourier feature generation in the forward pass
class SurvivalPredictionNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, fourier_order=1):
        super(SurvivalPredictionNN, self).__init__()
        self.fourier_order = fourier_order
        self.orders = torch.arange(1, fourier_order + 1).float().to(device)
        self.fc1 = nn.Linear(input_size * (2 * fourier_order + 1), hidden_size1)  # First layer with Fourier features
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, 2)  # Output layer now has 2 neurons for classification

    def forward(self, x):
        x = x.unsqueeze(-1)  # Add extra dimension for broadcasting
        fourier_features = torch.cat([torch.sin(self.orders * x), torch.cos(self.orders * x), x], dim=-1)  # Fourier series transformation
        fourier_features = fourier_features.view(x.shape[0], -1)  
        x = torch.relu(self.fc1(fourier_features))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)  # Output layer
        x = torch.softmax(x, dim=1)  # Apply softmax activation for classification
        return x


# Define the function to train the model with different subsets of data
# Use CrossEntropyLoss for classification tasks
criterion = nn.CrossEntropyLoss()

def RegressionNN(feature, data, train_sizes, batch_size=32, hidden_size1=64, hidden_size2=32, hidden_size3=16, lr=0.0001, num_epochs=50):
    print('Target Variable -', feature)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Separate the target column (A Stage) from the rest of the data
    X = data.drop(columns=feature)  # Features
    y = data[feature]  # Target
    
    # Convert target y to long type for CrossEntropyLoss
    y = y.astype(int)  # Ensure y is of integer type, not float

    # Initial split into 75% training data and 25% test data
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Convert test and validation data to PyTorch tensors and move to device
    X_test = torch.tensor(X_test.values.astype(np.float32)).to(device)
    y_test = torch.tensor(y_test.values.astype(int)).to(device)  # Ensure target is long for CrossEntropyLoss

    # Loop over different train sizes
    results = []
    for train_size in train_sizes:
        print(f'\nTraining with {train_size*100}% of the data:')
        current_train_size = int(len(X_train_val) * train_size)

        # Split the training data according to the current size
        X_train, _, y_train, _ = train_test_split(X_train_val, y_train_val, train_size=current_train_size, random_state=42)

        # Convert training data to PyTorch tensors and move to device
        X_train = torch.tensor(X_train.values.astype(np.float32)).to(device)
        y_train = torch.tensor(y_train.values.astype(int)).to(device)  # Ensure target is long for CrossEntropyLoss

        # Creating Data Loaders for training and test datasets
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        test_dataset = TensorDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Initializing the model and the optimizer
        input_size = X_train.shape[1]
        model = SurvivalPredictionNN(input_size, hidden_size1, hidden_size2, hidden_size3).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Training with validation
        for epoch in tqdm(range(num_epochs), desc=f"Epochs (Train Size: {train_size*100}%)"):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)

            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_labels in test_loader:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    val_outputs = model(val_inputs)
                    val_loss += criterion(val_outputs, val_labels).item() * val_inputs.size(0)
            val_loss /= len(test_loader.dataset)
            print(f'Validation Loss: {val_loss:.4f}')

        # Evaluate on test data after training
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        predictions = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                test_loss += criterion(outputs, labels).item() * inputs.size(0)
                
                # Get the predicted class by taking the argmax
                _, predicted = torch.max(outputs, 1)
                
                # Compare predicted class with true labels
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                predictions.extend(predicted.cpu().tolist())

        # Calculate the average loss
        test_loss /= len(test_loader.dataset)
        print(f'Test Loss: {test_loss:.4f}')

        # Calculate accuracy
        accuracy = correct / total * 100
        print(f'Accuracy: {accuracy:.2f}%')

        # Collect results
        results.append((train_size, accuracy))

    return results




# Define train sizes and target feature
train_sizes = [0.2, 0.4, 0.6, 0.8, 0.999668655]  # Proportions of the total data
feature = 'A Stage'  # Target column

# Load your data here
data = data_preprocessed_df  # Replace with your data source

# Call the function
results = RegressionNN(feature, data, train_sizes)


cuda
Target Variable - A Stage

Training with 20.0% of the data:


Epochs (Train Size: 20.0%):   0%|          | 0/50 [00:00<?, ?it/s]c:\ProgramData\miniconda3\envs\breastcancer1\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\ProgramData\miniconda3\envs\breastcancer1\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([11, 1])) that is different to the input size (torch.Size([11, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\ProgramData\miniconda3\envs\breastcancer1\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([19, 1])) that is different to the input size (torch.S

Validation Loss: 0.2504
Validation Loss: 0.2501
Validation Loss: 0.2500
Validation Loss: 0.2500
Validation Loss: 0.2500


Epochs (Train Size: 20.0%):  18%|█▊        | 9/50 [00:00<00:01, 24.71it/s]

Validation Loss: 0.2500
Validation Loss: 0.2500
Validation Loss: 0.2500
Validation Loss: 0.2500
Validation Loss: 0.2500
Validation Loss: 0.2500


...

Validation Loss: 0.2500
Validation Loss: 0.2500


Epochs (Train Size: 99.9668655%):  96%|█████████▌| 48/50 [00:07<00:00,  6.51it/s]

Validation Loss: 0.2500
Validation Loss: 0.2500


Epochs (Train Size: 99.9668655%): 100%|██████████| 50/50 [00:07<00:00,  6.53it/s]

Validation Loss: 0.2500
Validation Loss: 0.2500
Test Loss: 0.2500
Accuracy: 1423.46%
